In [1]:
import pickle
X_acl,X_ppg,y,y_participant,X_time = pickle.load(open('../data/tabular_data.p','rb'))

In [2]:
import numpy as np
X_ppg = X_ppg[:,np.arange(0,512,2),:]
X_all = np.concatenate([X_ppg,X_acl],axis=-1)

In [3]:
X_all.shape

(43363, 256, 8)

In [228]:
logo = LeaveOneGroupOut()
logo = LeavePGroupsOut(n_groups=2)
for train_index, test_index in logo.split(X_all, y, y_participant.reshape(-1)):
    train_x, test_x_unseen = X_all[train_index], X_all[test_index]
    train_y, test_y_unseen = y[train_index], y[test_index]
    train_participant, test_participant_unseen = y_participant[train_index], y_participant[test_index]
    train_participant, test_participant_unseen = X_time[train_index], X_time[test_index]
    break
train_x.shape,train_y.shape,test_y_unseen.shape,test_x_unseen.shape

((37544, 256, 3), (37544,), (5819,), (5819, 256, 3))

In [229]:
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer,LabelEncoder

participant_ids = train_participant.copy()
train_participant = OneHotEncoder().fit_transform(train_participant.reshape(-1,1)).todense()

In [230]:
train_x, test_x, train_y, test_y,train_participant, test_participant,participant_ids_train, participant_ids_test = train_test_split(train_x,
                                                            train_y,
                                                            train_participant,
                                                            participant_ids,
                                                            test_size = 0.2,
                                                            random_state=42,
                                                            stratify=participant_ids)

In [231]:
train_x, val_x, train_y, val_y, train_participant,val_participant,participant_ids_train, participant_ids_val  = train_test_split(train_x,
                                                            train_y,
                                                            train_participant,
                                                            participant_ids_train,
                                                            test_size = 0.1,
                                                            random_state=42,
                                                            stratify=participant_ids_train)

In [232]:
train_x.shape,test_x.shape,val_x.shape,train_participant.shape,test_participant.shape,val_participant.shape, 

((27031, 256, 3),
 (7509, 256, 3),
 (3004, 256, 3),
 (27031, 13),
 (7509, 13),
 (3004, 13))

In [233]:
from keras_self_attention import SeqSelfAttention
from keras import backend as K
from keras.layers import Concatenate, Flatten,LeakyReLU, Activation, RepeatVector, Permute, Multiply, Lambda, Dense, merge# Define a regular layer instead of writing a custom layer

In [234]:
def get_model(input_shape=(256,1),act='relu',loss="categorical_crossentropy",opt='adam',n_classes=350):
    model =  Sequential()
    model.add(Conv1D(100,10,input_shape=input_shape,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Conv1D(100,10,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Conv1D(100,10,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Conv1D(50,2,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Flatten())
    model.add(Dense(n_classes,activation='sigmoid',kernel_initializer='normal'))
    model.add(Dense(n_classes,activation='softmax',kernel_initializer='normal'))
    model.compile(loss=loss,optimizer=opt,metrics=['acc'])
    return model

In [235]:
n_classes = len(np.unique(participant_ids_train))
model = get_model(input_shape=(256,3),n_classes=n_classes)
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_48 (Conv1D)           (None, 256, 100)          3100      
_________________________________________________________________
max_pooling1d_48 (MaxPooling (None, 128, 100)          0         
_________________________________________________________________
batch_normalization_48 (Batc (None, 128, 100)          400       
_________________________________________________________________
dropout_36 (Dropout)         (None, 128, 100)          0         
_________________________________________________________________
conv1d_49 (Conv1D)           (None, 128, 100)          100100    
_________________________________________________________________
max_pooling1d_49 (MaxPooling (None, 64, 100)           0         
_________________________________________________________________
batch_normalization_49 (Batc (None, 64, 100)         

In [236]:
from keras.models import load_model
filepath = '../model_files/base_cnn_8_seconds_acl_filtered_wesad.hdf5'
# model = load_model(filepath)

In [242]:
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=40)
callbacks_list = [es,checkpoint]
history = model.fit(train_x,train_participant,validation_data=(val_x,val_participant), epochs=400, batch_size=200,
          callbacks=callbacks_list,shuffle=True)

Epoch 1/400
136/136 [==============================] - ETA: 0s - loss: 0.9999 - acc: 0.6443
Epoch 00001: val_loss improved from inf to 2.01921, saving model to ../model_files/base_cnn_8_seconds_acl_filtered_wesad.hdf5
136/136 [==============================] - 2s 14ms/step - loss: 0.9999 - acc: 0.6443 - val_loss: 2.0192 - val_acc: 0.4314
Epoch 2/400
136/136 [==============================] - ETA: 0s - loss: 0.9830 - acc: 0.6503
Epoch 00002: val_loss did not improve from 2.01921
136/136 [==============================] - 2s 12ms/step - loss: 0.9830 - acc: 0.6503 - val_loss: 6.7162 - val_acc: 0.2563
Epoch 3/400
136/136 [==============================] - ETA: 0s - loss: 0.9771 - acc: 0.6499
Epoch 00003: val_loss did not improve from 2.01921
136/136 [==============================] - 2s 12ms/step - loss: 0.9771 - acc: 0.6499 - val_loss: 4.0208 - val_acc: 0.3479
Epoch 4/400
136/136 [==============================] - ETA: 0s - loss: 0.9635 - acc: 0.6557
Epoch 00004: val_loss improved from 2.

Epoch 31/400
136/136 [==============================] - ETA: 0s - loss: 0.8376 - acc: 0.7027
Epoch 00031: val_loss did not improve from 1.63640
136/136 [==============================] - 2s 12ms/step - loss: 0.8376 - acc: 0.7027 - val_loss: 2.4825 - val_acc: 0.4451
Epoch 32/400
136/136 [==============================] - ETA: 0s - loss: 0.8242 - acc: 0.7072
Epoch 00032: val_loss did not improve from 1.63640
136/136 [==============================] - 2s 12ms/step - loss: 0.8242 - acc: 0.7072 - val_loss: 4.5028 - val_acc: 0.3462
Epoch 33/400
136/136 [==============================] - ETA: 0s - loss: 0.8124 - acc: 0.7158
Epoch 00033: val_loss did not improve from 1.63640
136/136 [==============================] - 2s 12ms/step - loss: 0.8124 - acc: 0.7158 - val_loss: 3.3485 - val_acc: 0.3802
Epoch 34/400
136/136 [==============================] - ETA: 0s - loss: 0.8160 - acc: 0.7095
Epoch 00034: val_loss did not improve from 1.63640
136/136 [==============================] - 2s 12ms/step - 

Epoch 62/400
136/136 [==============================] - ETA: 0s - loss: 0.7211 - acc: 0.7467
Epoch 00062: val_loss did not improve from 1.34566
136/136 [==============================] - 2s 12ms/step - loss: 0.7211 - acc: 0.7467 - val_loss: 4.0786 - val_acc: 0.3961
Epoch 63/400
136/136 [==============================] - ETA: 0s - loss: 0.7156 - acc: 0.7506
Epoch 00063: val_loss did not improve from 1.34566
136/136 [==============================] - 2s 12ms/step - loss: 0.7156 - acc: 0.7506 - val_loss: 2.4077 - val_acc: 0.4784
Epoch 64/400
136/136 [==============================] - ETA: 0s - loss: 0.7114 - acc: 0.7499
Epoch 00064: val_loss did not improve from 1.34566
136/136 [==============================] - 2s 12ms/step - loss: 0.7114 - acc: 0.7499 - val_loss: 2.1656 - val_acc: 0.4717
Epoch 65/400
136/136 [==============================] - ETA: 0s - loss: 0.7062 - acc: 0.7528
Epoch 00065: val_loss did not improve from 1.34566
136/136 [==============================] - 2s 12ms/step - 

Epoch 93/400
136/136 [==============================] - ETA: 0s - loss: 0.6428 - acc: 0.7739
Epoch 00093: val_loss did not improve from 1.12339
136/136 [==============================] - 2s 12ms/step - loss: 0.6428 - acc: 0.7739 - val_loss: 5.9145 - val_acc: 0.3402
Epoch 94/400
136/136 [==============================] - ETA: 0s - loss: 0.6475 - acc: 0.7745
Epoch 00094: val_loss did not improve from 1.12339
136/136 [==============================] - 2s 12ms/step - loss: 0.6475 - acc: 0.7745 - val_loss: 2.2309 - val_acc: 0.5210
Epoch 95/400
136/136 [==============================] - ETA: 0s - loss: 0.6437 - acc: 0.7718
Epoch 00095: val_loss did not improve from 1.12339
136/136 [==============================] - 2s 12ms/step - loss: 0.6437 - acc: 0.7718 - val_loss: 4.5570 - val_acc: 0.3589
Epoch 96/400
136/136 [==============================] - ETA: 0s - loss: 0.6490 - acc: 0.7743
Epoch 00096: val_loss did not improve from 1.12339
136/136 [==============================] - 2s 12ms/step - 

In [239]:
model = load_model(filepath)
test_participant_pred = model.predict(test_x)

In [240]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [241]:
print(confusion_matrix(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
print(accuracy_score(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
#filtered_acl

[[139   3   8  52   1   6   9  13  18  92 180   2   1]
 [  0 350   0  11  63   1   0   1   8   4   9   0 201]
 [ 11   0 133  63   0   5   5  16   8  27 248   6   1]
 [  1   1   0 445   0   3   0   0  26  60 167   3   0]
 [  0  62   0   1 265   0   1   0   9   2   7   0 244]
 [ 17   1   1  22   2 196   3   2  26 139 114  22   8]
 [ 20   0   4   7   0   6  90  10   4 136 270   2   0]
 [ 17   1   8  39   0   8  10 118   8  92 305  19   0]
 [  2   0   4  74   0   2   0   2 251  98 124   5   0]
 [  6   1   2  53   0  10   1   4  31 253 128  32   0]
 [  0   4   0  28  10   2   6   1  11  90 388   5   1]
 [  0   0   0  33   0  10   0   4  41 114 184 221   0]
 [  0  69   0   6 106   1   0   0  12   3   3   0 354]]
0.42655480090557996


In [222]:
print(confusion_matrix(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
print(accuracy_score(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
#filtered ppg+filtered_acl

[[410   5  19   4  12   4  17   6  24   6   1   6  10]
 [  0 575   6   5  46   0   0   1  10   0   3   0   2]
 [  8  23 436   6   6   1   3   6  13  10   0  10   1]
 [  3  10   2 592  13   2   7   1  27  17   8  21   3]
 [  3  13   7   3 515   0  15   2   7   2  14   2   8]
 [  7   0   1   0   2 490  13   1   1   5   8   5  20]
 [  7   2   3   6   3  14 434   0  18  21   3   0  38]
 [  5   0  15   3  10   7   2 548   0   3  12  20   0]
 [ 16  17  15  26   6   0  18   0 448   4   2   0  10]
 [  4   0  18   9   0  11   9   2   6 453   2   6   1]
 [  3   0   0   7  14  19   8   3   2   6 421  25  38]
 [  8   4  31  17   6   6   1   8   5  20  18 477   6]
 [  2   0   0   3   4  14  21   0  11   1   8   0 490]]
0.8375282993740845


In [176]:
print(confusion_matrix(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
print(accuracy_score(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
#filtered ppg+acl

[[496   0   5   3   0   2   4   1   4   3   0   6   0]
 [  0 644   0   1   2   0   0   0   0   0   0   0   1]
 [  1   0 510   2   0   0   0   1   4   0   5   0   0]
 [  1   0   1 691   0   4   0   0   2   5   0   2   0]
 [  0  32   0   0 556   0   0   0   0   0   0   0   3]
 [  1   0   0   3   0 535   3   0   1   1   7   2   0]
 [  7   0   2   0   0   7 526   1   3   3   0   0   0]
 [ 13   0   5   2   0   1   0 592   1   0   5   6   0]
 [  8   0   6   5   0   1   1   1 536   0   3   1   0]
 [  0   0   5   3   0   0   2   0   0 504   5   2   0]
 [  1   0   0   4   0   4   2   4   0   3 523   5   0]
 [  1   0   1   9   0   4   2   6   3   1   1 579   0]
 [  0   0   0   0   5   0   0   0   0   0   0   0 549]]
0.9643094952723399


In [194]:
print(confusion_matrix(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
print(accuracy_score(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
#filtered ppg

[[419   5  19   2  15   1  11  11  34   0   2   3   2]
 [  6 584   9   5  32   0   4   2   4   0   1   0   1]
 [ 12  24 428   4   6   2   1  15   6   9   2  11   3]
 [ 12   6   3 593  16   4   6   1  29  12   7   9   8]
 [  6  37   1   7 506   0   9   4   7   0   5   3   6]
 [ 11   0   2   1   2 486  17   4   1   6   8   3  12]
 [ 15   1   0   7   7  25 437   1  19  11   1   1  24]
 [  7   3   8   9  16   4   1 544   0   2  15  15   1]
 [ 25  18  16  24   3   0  14   1 446   2   0   1  12]
 [ 18   1   8   9   0  22   6   3   6 432   0  12   4]
 [  6   2   1  13  12  10   4   8   3   1 444  25  17]
 [ 16   4  17  18   5   4   3  23   3   8  24 478   4]
 [ 11   1   1   3   5  17  12   0  17   1  11   0 475]]
0.8352643494473299


In [94]:
print(confusion_matrix(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
print(accuracy_score(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
#ppg+acl

[[504   0   0   1   0   1   2   5   0   1   1   9   0]
 [  0 644   0   0   2   0   0   0   0   0   0   1   1]
 [  8   0 488   4   0   1   0   6   8   4   1   3   0]
 [  1   0   0 689   0   4   0   0   3   3   1   5   0]
 [  0   7   0   0 581   0   0   0   0   0   0   0   3]
 [  0   0   0   0   0 545   1   4   2   0   1   0   0]
 [  2   0   0   0   0  10 531   0   2   4   0   0   0]
 [  5   0   0   0   0   0   3 608   1   3   0   5   0]
 [  3   3   2   3   0   2   4   1 544   0   0   0   0]
 [  1   0   1   2   0   1   1   1   0 510   1   3   0]
 [  0   0   0   3   0   6   0   0   0   1 534   2   0]
 [  1   0   0   4   0   1   0   7   0   1   1 592   0]
 [  0   0   0   0   3   0   0   0   0   0   0   0 551]]
0.9749633772805966


In [110]:
print(confusion_matrix(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
print(accuracy_score(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
#ppg

[[473   1  12   0   8   3   3   2  11   2   2   7   0]
 [  0 631   2   1   4   0   0   1   5   0   3   0   1]
 [  2   7 491   2   7   0   0   1   7   2   2   2   0]
 [  1   1   1 678   0   3   0   0  14   1   6   0   1]
 [  1  10   0   0 572   0   0   0   3   0   1   4   0]
 [  0   0   0   5   0 535   5   1   0   1   1   0   5]
 [  7   0   1   1   0   5 515   0  14   2   0   0   4]
 [  4   0   6   0   7   2   2 579   1   1   3  20   0]
 [  2   1   3   3   1   1   3   0 547   0   0   1   0]
 [  4   1   6  12   0   0   5   0   1 486   0   5   1]
 [  2   3   0   4   2   4   1   2   3   1 508   4  12]
 [  2   1   6   2   6   0   0   1   3   1   2 583   0]
 [  0   0   0   3   0   6   0   0   2   0   2   0 541]]
0.9507257957118125


In [131]:
print(confusion_matrix(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
print(accuracy_score(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
#acl

[[414   0  14  18   0  26   7   7   6  17   7   8   0]
 [  0 592   0   0  32   0   0   0   0   0   0   0  24]
 [  5   0 454  12   0  11  10   5  12   7   4   3   0]
 [  4   1   4 641   3   5   7   5   1  12   6  17   0]
 [  0  29   0   0 519   0   0   0   0   0   0   0  43]
 [ 15   0  17  13   0 444  15  10  10   8   8  13   0]
 [ 32   0  10   4   0  16 449   9  15   8   5   1   0]
 [ 22   0   3  14   0  10  13 489   8  41   8  17   0]
 [  7   0  16   5   0  11  20   2 445  26  14  16   0]
 [  9   0  20   5   0   6   9  15  28 419   4   6   0]
 [  4   3   9  10   0   8   3  18   8   2 472   9   0]
 [  7   0   0  20   0   8  12   6  17   8   6 523   0]
 [  0  35   0   1  16   0   0   0   0   0   0   0 502]]
0.8473831402317219
